In [1]:
from graph_of_thoughts.prompter import LanguageModelPrompter
from graph_of_thoughts.language_model import ChatGPT
lm = ChatGPT()
table = """ {
        "column_names": [
            [
                -1,
                "*"
            ],
            [
                0,
                "city id"
            ],
            [
                0,
                "official name"
            ],
            [
                0,
                "status"
            ],
            [
                0,
                "area km 2"
            ],
            [
                0,
                "population"
            ],
            [
                0,
                "census ranking"
            ],
            [
                1,
                "farm id"
            ],
            [
                1,
                "year"
            ],
            [
                1,
                "total horses"
            ],
            [
                1,
                "working horses"
            ],
            [
                1,
                "total cattle"
            ],
            [
                1,
                "oxen"
            ],
            [
                1,
                "bulls"
            ],
            [
                1,
                "cows"
            ],
            [
                1,
                "pigs"
            ],
            [
                1,
                "sheep and goats"
            ],
            [
                2,
                "competition id"
            ],
            [
                2,
                "year"
            ],
            [
                2,
                "theme"
            ],
            [
                2,
                "host city id"
            ],
            [
                2,
                "hosts"
            ],
            [
                3,
                "competition id"
            ],
            [
                3,
                "farm id"
            ],
            [
                3,
                "rank"
            ]
        ],
        "column_names_original": [
            [
                -1,
                "*"
            ],
            [
                0,
                "City_ID"
            ],
            [
                0,
                "Official_Name"
            ],
            [
                0,
                "Status"
            ],
            [
                0,
                "Area_km_2"
            ],
            [
                0,
                "Population"
            ],
            [
                0,
                "Census_Ranking"
            ],
            [
                1,
                "Farm_ID"
            ],
            [
                1,
                "Year"
            ],
            [
                1,
                "Total_Horses"
            ],
            [
                1,
                "Working_Horses"
            ],
            [
                1,
                "Total_Cattle"
            ],
            [
                1,
                "Oxen"
            ],
            [
                1,
                "Bulls"
            ],
            [
                1,
                "Cows"
            ],
            [
                1,
                "Pigs"
            ],
            [
                1,
                "Sheep_and_Goats"
            ],
            [
                2,
                "Competition_ID"
            ],
            [
                2,
                "Year"
            ],
            [
                2,
                "Theme"
            ],
            [
                2,
                "Host_city_ID"
            ],
            [
                2,
                "Hosts"
            ],
            [
                3,
                "Competition_ID"
            ],
            [
                3,
                "Farm_ID"
            ],
            [
                3,
                "Rank"
            ]
        ],
        "column_types": [
            "text",
            "number",
            "text",
            "text",
            "number",
            "number",
            "text",
            "number",
            "number",
            "number",
            "number",
            "number",
            "number",
            "number",
            "number",
            "number",
            "number",
            "number",
            "number",
            "text",
            "number",
            "text",
            "number",
            "number",
            "number"
        ],
        "db_id": "farm",
        "foreign_keys": [
            [
                20,
                1
            ],
            [
                23,
                7
            ],
            [
                22,
                17
            ]
        ],
        "primary_keys": [
            1,
            7,
            17,
            22
        ],
        "table_names": [
            "city",
            "farm",
            "farm competition",
            "competition record"
        ],
        "table_names_original": [
            "city",
            "farm",
            "farm_competition",
            "competition_record"
        ]
    }"""
input = 'Show the official names of the cities that have hosted more than one competition.'
main_task = f'Write a SQL include SELECT, FROM, WHERE to answer the specific QUESTION knowing Table = {table}\nQUESTION = {input}'
prompter = LanguageModelPrompter(lm, f"Main Task: {main_task}", json_format=False)
print(f"{main_task} {input}")




Write a SQL include SELECT, FROM, WHERE to answer the specific QUESTION knowing Table =  {
        "column_names": [
            [
                -1,
                "*"
            ],
            [
                0,
                "city id"
            ],
            [
                0,
                "official name"
            ],
            [
                0,
                "status"
            ],
            [
                0,
                "area km 2"
            ],
            [
                0,
                "population"
            ],
            [
                0,
                "census ranking"
            ],
            [
                1,
                "farm id"
            ],
            [
                1,
                "year"
            ],
            [
                1,
                "total horses"
            ],
            [
                1,
                "working horses"
            ],
            [
                1,
               

In [2]:
split_prompt = prompter.split_prompt({"state": f"Start\nInput", "num_split": 3, "current": input})
print(split_prompt)

<Instruction>STEP: Divide the current task into three subtasks by focusing on the following aspects:
1. Identifying the cities that have hosted competitions.
2. Counting the number of competitions hosted by each city.
3. Selecting the cities that have hosted more than one competition and showing their official names. 
Answer each subtask seperately in json format with index, example {"0":...,"1":...}without any addtional text or thoughts.</Instruction>
Input: Show the official names of the cities that have hosted more than one competition.
Answer:


In [3]:
split_result = lm.get_response_texts(lm.query(split_prompt, 1, main_task))[0]
print(split_result)


```json
{
    "0": {
        "query": "SELECT Host_city_ID, COUNT(*) AS num_competitions FROM farm_competition GROUP BY Host_city_ID",
        "explanation": "Identifying the cities that have hosted competitions and counting the number of competitions hosted by each city."
    },
    "1": {
        "query": "SELECT city.Official_Name FROM city JOIN (SELECT Host_city_ID, COUNT(*) AS num_competitions FROM farm_competition GROUP BY Host_city_ID HAVING COUNT(*) > 1) AS hosted_competitions ON city.City_ID = hosted_competitions.Host_city_ID",
        "explanation": "Selecting the cities that have hosted more than one competition and showing their official names."
    }
}
```


In [4]:
split_result = split_result.replace('`', '')
split_result = split_result.replace('json', '')

In [5]:
print(split_result)


{
    "0": {
        "query": "SELECT Host_city_ID, COUNT(*) AS num_competitions FROM farm_competition GROUP BY Host_city_ID",
        "explanation": "Identifying the cities that have hosted competitions and counting the number of competitions hosted by each city."
    },
    "1": {
        "query": "SELECT city.Official_Name FROM city JOIN (SELECT Host_city_ID, COUNT(*) AS num_competitions FROM farm_competition GROUP BY Host_city_ID HAVING COUNT(*) > 1) AS hosted_competitions ON city.City_ID = hosted_competitions.Host_city_ID",
        "explanation": "Selecting the cities that have hosted more than one competition and showing their official names."
    }
}



In [6]:
import json
split_result_dict = json.loads(split_result)

In [8]:
aggregate_prompt = prompter.aggregate_prompt({"state": """STEP: Divide the current task into three subtasks by focusing on the following aspects:
1. Identifying the cities that have hosted competitions.
2. Counting the number of competitions hosted by each city.
3. Selecting the cities that have hosted more than one competition and showing their official names.""", "current":split_result})


In [9]:
print(aggregate_prompt)

<Instruction>STEP: Use a subquery to count the number of competitions hosted by each city and filter the result to only include cities that have hosted more than one competition. Then, join this result with the city table using the city id to retrieve the official names of the cities that meet the criteria. Finally, select the official names of these cities to obtain the final result. 
Only answer the result in json format with index, example {"0":...,"1":...}without any addtional text or thoughts.</Instruction>
Input: 
{
    "0": {
        "query": "SELECT Host_city_ID, COUNT(*) AS num_competitions FROM farm_competition GROUP BY Host_city_ID",
        "explanation": "Identifying the cities that have hosted competitions and counting the number of competitions hosted by each city."
    },
    "1": {
        "query": "SELECT city.Official_Name FROM city JOIN (SELECT Host_city_ID, COUNT(*) AS num_competitions FROM farm_competition GROUP BY Host_city_ID HAVING COUNT(*) > 1) AS hosted_compe

In [10]:
lm.get_response_texts(lm.query(aggregate_prompt, 1, main_task))[0]

'{\n    "0": "City 1",\n    "1": "City 2",\n    "2": "City 3"\n}'